In [1]:
"""
This cell is used to import the required libraries and set up the PSSE environment,
this is a bit more complex than it should because of the way I developed WEC-GRID
"""
# import PSSE stuff
# Standard Libraries
import os
import sys
import time

# # local libraries
# from WEC_GRID.utilities.util import read_paths
# from WEC_GRID.viz.psse_viz import PSSEVisualizer

# # Initialize the PATHS dictionary
# PATHS = read_paths()
# CURR_DIR = os.path.dirname(os.path.abspath("./"))

# os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
# psse_path = PATHS["psse"]
# sys.path.extend(
#     [
#         os.path.join(psse_path, subdir)
#         for subdir in ["PSSPY37", "PSSBIN", "PSSLIB", "EXAMPLE"]
#     ]
# )
# os.environ["PATH"] = (
#     os.path.join(psse_path, "PSSPY37")
#     + ";"
#     + os.path.join(psse_path, "PSSBIN")
#     + ";"
#     + os.path.join(psse_path, "EXAMPLE")
#     + ";"
#     + os.environ["PATH"]
# )

import psse35

psse35.set_minor(3)
import psspy
import dyntools

psspy.psseinit(50)


_i = psspy.getdefaultint()
_f = psspy.getdefaultreal()
_s = psspy.getdefaultchar()

import matplotlib.pyplot as plt
import numpy as np
import dyntools

    Sets PSSE environment to latest minor version among installed versions of PSSE 35.
    Use psse35.set_minor(n) to set PSSE35 minor version (n) to use.
        Example, for PSSE 35.0.x, use this as: psse35.set_minor(0)

 PSS(R)E Xplore Version 35
 Copyright (c) 1976-2025
 Siemens Industry, Inc.,
 Power Technologies International                            (PTI)
 This program is a confidential  unpublished  work  created  and  first
 licensed in 1976.  It is a trade secret which is the property of  PTI.
 All use,  disclosure,  and/or reproduction not specifically authorized
 by  PTI  is prohibited.   This  program is protected  under  copyright
 laws  of  non-U.S.  countries  and  by  application  of  international
 treaties.  All  Rights  Reserved  Under  The  Copyright  Laws.

 Could not find entry point runpy_add_to_sys_path

           SIEMENS POWER TECHNOLOGIES INTERNATIONAL

    50 BUS POWER SYSTEM SIMULATOR--PSS(R)E Xplore-35.5.2

             INITIATED ON MON, JAN 27 2025  20

In [2]:
case_file = "../WecGrid/models/grid_models/IEEE_24_bus.RAW"
psspy.read(1,case_file)




 Reading IC, SBASE, REV, XFRRAT, NXFRAT, BASFRQ...

 Converting RAW data from PSS(R)E 33 to PSS(R)E 34 format
 Starting processing of Case identification data records
 Finished processing of 3 records for Case identification data
 Starting processing of Bus data records
 Finished processing of 24 records for Bus data
 Starting processing of Load data records
 Finished processing of 16 records for Load data
 Starting processing of Fixed shunt data records
 Finished processing of 0 records for Fixed shunt data
 Starting processing of Generator data records
 Finished processing of 11 records for Generator data
 Starting processing of Branch data records
 Finished processing of 32 records for Branch data
 Finished processing of 0 records for System switching device data
 Starting processing of Transformer data records
 Finished processing of 24 records for 6 Transformer data
 Starting processing of Area data records
 Finished processing of 1 records for Area data
 Starting processing of T

0

In [3]:
psspy.fnsl()


 Ordering network...
 Diagonals = 23   Off-diagonals = 45   Maximum size = 62

  ITER       DELTAP      BUS         DELTAQ      BUS        DELTA/V/      BUS       DELTAANG      BUS
   0         0.0004(     19     )    0.0110(      5     )
                                                             0.00056(      7     )   0.00042(     21     )
   1         0.0000(     23     )    0.0000(     21     )
                                                             0.00000(      7     )   0.00000(     21     )
   2         0.0000(     23     )    0.0000(      6     )


 Reached tolerance in 2 iterations

 Largest mismatch:      0.00 MW      0.00 Mvar      0.00 MVA at bus 22 [22          230.00]
 System total absolute mismatch:                    0.00 MVA

 SWING BUS SUMMARY:
   BUS#-SCT X-- NAME --X BASKV      PGEN     PMAX    PMIN      QGEN     QMAX    QMIN
      1     1           138.00      35.9   1000.0     0.0     180.5   9900.0 -9900.0


0

In [14]:
ierr, pg_list = psspy.amachreal(-1, 1,"PGEN")
print(pg_list[0])
ierr, qg_list = psspy.amachreal(-1, 1,"QGEN")
print(qg_list[0])


[35.93006896972656, 67.0, 64.0, 200.0, 0.0, 274.0, 245.0, 144.0, 294.0, 150.0, 200.0]
[180.48858642578125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 199.4558563232422, 0.0]


In [4]:
psspy.bus_data_2(2, flag=2)


 Keyword arguments not used: flag=2

 No power flow data changed for bus 2 [2           138.00]


0

In [4]:

# Bus information
ibus = 25
inode = 0
bus_name = "TESTBUS"

# Step 1: Add the bus
intgar_bus = [1, 1, 1, 1]
#realar_bus = [0.0, 1.0, 0.0, 1.1, 0.9, 1.1, 0.9]
realar_bus = [230.0, 1.0, 0.0, 1.1, 0.9, 1.1, 0.9]
ierr = psspy.bus_data_4(ibus, inode, intgar_bus, realar_bus, bus_name)
if ierr == 0:
    print(f"Bus {ibus} added successfully.")
else:
    print(f"Error adding bus {ibus}. PSS®E error code: {ierr}")
    exit()

# Step 2: Add plant data
intgar_plant = [0, 0]
realar_plant = [1.0, 100.0]
ierr = psspy.plant_data_4(ibus, inode, intgar_plant, realar_plant)
if ierr == 0:
    print(f"Plant data added successfully to bus {ibus}.")
else:
    print(f"Error adding plant data to bus {ibus}. PSS®E error code: {ierr}")
    exit()

# Step 3: Add the generator
gen_id = "1"
gen_name = ""
intgar_gen = [1, 1, 0, 0, 0, 0, 0]
realar_gen = [
    0.0,    # PG
    0.0,    # QG
    9999.0, # QT
   -9999.0, # QB
    9999.0, # PT
   -9999.0, # PB
    100.0,  # MBASE
    0.0,    # ZR
    1.0,    # ZX
    0.0,    # RT
    0.0,    # XT
    1.0,    # GTAP
    1.0,    # F1
    0.0,    # F2
    0.0,    # F3
    0.0,    # F4
    0.0     # WPF
]
ierr = psspy.machine_data_4(ibus, gen_id, intgar_gen, realar_gen, gen_name)
if ierr == 0:
    print(f"Generator {gen_id} added successfully to bus {ibus}.")
else:
    print(f"Error adding generator {gen_id} to bus {ibus}. PSS®E error code: {ierr}")
    exit()

# Define the branch details
from_bus = 24  # Starting bus
to_bus = 25    # Ending bus

# Branch name (leave blank if not needed)
branch_name = ""

# Add the branch
ierr = psspy.branch_data_3(
    from_bus, to_bus)
if ierr == 0:
    print(f"Branch from {from_bus} to {to_bus} added successfully.")
else:
    print(f"Error adding branch from {from_bus} to {to_bus}. PSS®E error code: {ierr}")


 Bus 25 [TESTBUS     230.00] added. Power flow data items with non-default values:
 X--DEFAULT---X  X---ACTUAL---X  DATA ITEM
    0.00000         230.000      BASKV
 "            "  "TESTBUS     "  NAME
Bus 25 added successfully.

 Plant at bus 25 [TESTBUS     230.00] added. Data items with non-default values:
 X--DEFAULT---X  X---ACTUAL---X  DATA ITEM
          0              25      IREG
Plant data added successfully to bus 25.

 Machine "1" at bus 25 [TESTBUS     230.00] added. Power flow data items with non-default values:
 X--DEFAULT---X  X---ACTUAL---X  DATA ITEM
    1.00000         0.00000      WPF
Generator 1 added successfully to bus 25.

 Non-transformer branch circuit "1" from 24 [24          230.00] to 25 [TESTBUS     230.00] added with default data
Branch from 24 to 25 added successfully.


In [4]:

# Bus information
ibus = 25  # New bus number
inode = 0  # Node section number
bus_name = "TESTBUS"  # Bus name

# Add the bus
intgar_bus = [1, 1, 1, 1]  # Default values for bus type, area, zone, owner
realar_bus = [0.0, 1.0, 0.0, 1.1, 0.9, 1.1, 0.9]  # Default voltage and limits
ierr = psspy.bus_data_4(ibus, inode, intgar_bus, realar_bus, bus_name)
if ierr == 0:
    print(f"Bus {ibus} added successfully.")
else:
    print(f"Error adding bus {ibus}. PSS®E error code: {ierr}")
    exit()

# Add plant data
intgar_plant = [0, 0]  # Default: no remote or regulated bus
realar_plant = [1.0, 100.0]  # Scheduled voltage = 1.0, reactive power contribution = 100%
ierr = psspy.plant_data_4(ibus, inode, intgar_plant, realar_plant)
if ierr == 0:
    print(f"Plant data added successfully to bus {ibus}.")
else:
    print(f"Error adding plant data to bus {ibus}. PSS®E error code: {ierr}")
    exit()

# Add the generator
gen_id = "1"  # Generator identifier
gen_name = ""  # Leave blank to avoid droop control errors
intgar_gen = [1, 1, 0, 0, 0, 0, 0]  # In-service, owner 1, conventional machine
realar_gen = [
    0.0,  # PG: Active power output
    0.0,  # QG: Reactive power output
    9999.0,  # QT: Reactive power upper limit
    -9999.0,  # QB: Reactive power lower limit
    9999.0,  # PT: Active power upper limit
    -9999.0,  # PB: Active power lower limit
    100.0,  # MBASE: Machine MVA base
    0.0,  # ZR: Resistance
    1.0,  # ZX: Reactance
    0.0,  # RT: Transformer resistance
    0.0,  # XT: Transformer reactance
    1.0,  # GTAP: Transformer tap ratio
    1.0,  # F1: First owner fraction
    0.0,  # F2: Second owner fraction
    0.0,  # F3: Third owner fraction
    0.0,  # F4: Fourth owner fraction
    1.0,  # WPF: Power factor
]
ierr = psspy.machine_data_4(ibus, gen_id, intgar_gen, realar_gen, gen_name)
if ierr == 0:
    print(f"Generator {gen_id} added successfully to bus {ibus}.")
else:
    print(f"Error adding generator {gen_id} to bus {ibus}. PSS®E error code: {ierr}")


 Bus 25 [TESTBUS           ] added. Power flow data items with non-default values:
 X--DEFAULT---X  X---ACTUAL---X  DATA ITEM
 "            "  "TESTBUS     "  NAME
Bus 25 added successfully.

 Plant at bus 25 [TESTBUS           ] added. Data items with non-default values:
 X--DEFAULT---X  X---ACTUAL---X  DATA ITEM
          0              25      IREG
Plant data added successfully to bus 25.

 Machine "1" at bus 25 [TESTBUS           ] added with default data
Generator 1 added successfully to bus 25.


In [5]:

# Bus information
ibus = 25
inode = 0
bus_name = "TESTBUS"

# Step 1: Add the bus
intgar_bus = [1, 1, 1, 1]
realar_bus = [0.0, 1.0, 0.0, 1.1, 0.9, 1.1, 0.9]
ierr = psspy.bus_data_4(ibus, inode, intgar_bus, realar_bus, bus_name)
if ierr == 0:
    print(f"Bus {ibus} added successfully.")
else:
    print(f"Error adding bus {ibus}. PSS®E error code: {ierr}")
    exit()

# Step 2: Add plant data
intgar_plant = [0, 0]
realar_plant = [1.0, 100.0]
ierr = psspy.plant_data_4(ibus, inode, intgar_plant, realar_plant)
if ierr == 0:
    print(f"Plant data added successfully to bus {ibus}.")
else:
    print(f"Error adding plant data to bus {ibus}. PSS®E error code: {ierr}")
    exit()

# Step 3: Add the generator
gen_id = "1"
gen_name = ""
intgar_gen = [1, 1, 0, 0, 0, 0, 0]
realar_gen = [
    0.0,    # PG
    0.0,    # QG
    9999.0, # QT
   -9999.0, # QB
    9999.0, # PT
   -9999.0, # PB
    100.0,  # MBASE
    0.0,    # ZR
    1.0,    # ZX
    0.0,    # RT
    0.0,    # XT
    1.0,    # GTAP
    1.0,    # F1
    0.0,    # F2
    0.0,    # F3
    0.0,    # F4
    0.0     # WPF
]
ierr = psspy.machine_data_4(ibus, gen_id, intgar_gen, realar_gen, gen_name)
if ierr == 0:
    print(f"Generator {gen_id} added successfully to bus {ibus}.")
else:
    print(f"Error adding generator {gen_id} to bus {ibus}. PSS®E error code: {ierr}")
    exit()

# Step 4: Add a transmission line
from_bus = 24  # Connect from bus 1 (existing bus) to the new bus
ckt_id = "1"  # Circuit ID for the line

intgar_line = [1, 1, 1]  # Branch parameters: Owner, status, meter location
realar_line = [
    0.01,  # Resistance (R) in p.u.
    0.1,   # Reactance (X) in p.u.
    0.0,   # Line charging (B) in p.u.
    100.0, # Line MVA rating 1
    0.0,   # Line MVA rating 2
    0.0,   # Line MVA rating 3
]

ierr = psspy.branch_data(from_bus, ibus, ckt_id, intgar_line, realar_line)
if ierr == 0:
    print(f"Line added successfully from bus {from_bus} to bus {ibus}.")
else:
    print(f"Error adding line from bus {from_bus} to bus {ibus}. PSS®E error code: {ierr}")


 No power flow data changed for bus 25 [TESTBUS           ]
Bus 25 added successfully.

 No plant data changed at bus 25 [TESTBUS           ]
Plant data added successfully to bus 25.

 Power flow data changed for machine "1" at bus 25 [TESTBUS           ]:
 X--ORIGINAL--X  X-NEW VALUE--X  DATA ITEM
    1.00000         0.00000      WPF
Generator 1 added successfully to bus 25.

 Messages for api BRANCH_DATA_3
 Messages for circuit "1" from 1 [1           138.00] to 25 [TESTBUS           ]:   (000510)
  Warn: Non-transformer branch connects buses with different base voltages   (000526)

 Non-transformer branch ckt "1" from 1 [1           138.00] to 25 [TESTBUS           ] added. Power flow data items with non-default values:
 X----------------DEFAULT-----------------X  X-----------------ACTUAL-----------------X  DATA ITEM
                  0.00000                                    0.100000E-01                R
                 0.100000E-03                                0.100000       

In [10]:
psspy.machine_data_2(7, "1", realar1=0.0)


 Power flow data changed for machine "1" at bus 7 [7           138.00]:
 X--ORIGINAL--X  X-NEW VALUE--X  DATA ITEM
    64.0000         0.00000      PG


0

In [11]:
psspy.bus_data_4(25, intgar1=1, realar1=1.0) 


 Messages for api BUS_DATA_4
 Messages for bus 25:   (000612)
 FATAL: INode = <omitted value> but must be 0 for a bus that is not in the case   (000631)


9

In [14]:
intgar = [1,  # IDE: Bus type code (1 by default, slack bus)
          1,  # AREA: Area number (1 by default)
          1,  # ZONE: Zone number (1 by default)
          1]  # OWNER: Owner number (1 by default)


realar = [0.0,  # BASKV: Bus base voltage in kV (0.0 by default)
          1.0,  # VM: Bus voltage magnitude in p.u. (1.0 by default)
          0.0,  # VA: Bus voltage phase angle in degrees (0.0 by default)
          1.1,  # NMAXV: Normal bus voltage high limit in p.u. (1.1 by default)
          0.9,  # NMINV: Normal bus voltage low limit in p.u. (0.9 by default)
          1.1,  # EMAXV: Emergency bus voltage high limit in p.u. (1.1 by default)
          0.9]  # EMINV: Emergency bus voltage low limit in p.u. (0.9 by default)


ibus = 25  # New bus number
inode = 0  # Node section number
name = "TestBus"  # Bus name


# Call bus_data_4 API
ierr = psspy.bus_data_4(ibus, inode, intgar, realar, name)
if ierr == 0:
    print(f"Bus {ibus} added successfully.")
else:
    print(f"Error adding bus {ibus}. PSS®E error code: {ierr}")
    
    

intgar = [1,  # STAT: Machine status (1 = in-service by default)
          1,  # O1: First owner number (default is 1)
          0,  # O2: Second owner number (default is 0)
          0,  # O3: Third owner number (default is 0)
          0,  # O4: Fourth owner number (default is 0)
          0,  # WMOD: Reactive power limits mode (0 = conventional machine)
          0]  # BASFLG: Baseload machine flag (0 = not baseload)

realar = [0.0,    # PG: Machine active power output (0.0 by default)
          0.0,    # QG: Machine reactive power output (0.0 by default)
          9999.0, # QT: Reactive power upper limit (9999.0 by default)
         -9999.0, # QB: Reactive power lower limit (-9999.0 by default)
          9999.0, # PT: Active power upper limit (9999.0 by default)
         -9999.0, # PB: Active power lower limit (-9999.0 by default)
          100.0,  # MBASE: Machine MVA base (100.0 by default)
          0.0,    # ZR: Machine resistance (0.0 by default)
          1.0,    # ZX: Machine reactance (1.0 by default)
          0.0,    # RT: Step-up transformer resistance (0.0 by default)
          0.0,    # XT: Step-up transformer reactance (0.0 by default)
          1.0,    # GTAP: Step-up transformer tap ratio (1.0 by default)
          1.0,    # F1: First owner fraction (1.0 by default)
          0.0,    # F2: Second owner fraction (0.0 by default)
          0.0,    # F3: Third owner fraction (0.0 by default)
          0.0,    # F4: Fourth owner fraction (0.0 by default)
          1.0]    # WPF: Non-conventional machine power factor (1.0 by default)

gen_id = '1'  # Generator identifier
gen_name = "TestGenerator"  # Generator name

# Call machine_data_4 API to add the generator
ierr = psspy.machine_data_4(ibus, gen_id, intgar, realar, gen_name)
if ierr == 0:
    print(f"Generator {gen_id} added successfully to bus {ibus}.")
else:
    print(f"Error adding generator {gen_id} to bus {ibus}. PSS®E error code: {ierr}")



 No power flow data changed for bus 25 [TESTBUS           ]
Bus 25 added successfully.

 Messages for api MACHINE_DATA_4
 Messages for machine "1" at bus 25 [TESTBUS           ]:   (002601)
 FATAL: No plant data at this bus   (002595)
Error adding generator 1 to bus 25. PSS®E error code: 3


In [ ]:
import psspy

ibus = 25  # New bus number
inode = 0  # Node section number
bus_name = "TESTBUS"  # Bus name

# Add the bus
intgar_bus = [1, 1, 1, 1]
realar_bus = [0.0, 1.0, 0.0, 1.1, 0.9, 1.1, 0.9]
ierr = psspy.bus_data_4(ibus, inode, intgar_bus, realar_bus, bus_name)
if ierr == 0:
    print(f"Bus {ibus} added successfully.")
else:
    print(f"Error adding bus {ibus}. PSS®E error code: {ierr}")

# Add plant data
ierr = psspy.plant_data(ibus, intgar=[0], realar=[0.0])
if ierr == 0:
    print(f"Plant data added successfully to bus {ibus}.")
else:
    print(f"Error adding plant data to bus {ibus}. PSS®E error code: {ierr}")

# Add the generator
intgar_gen = [1, 1, 0, 0, 0, 0, 0]
realar_gen = [0.0, 0.0, 9999.0, -9999.0, 9999.0, -9999.0, 100.0, 0.0, 1.0,
              0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0]
gen_id = '1'
gen_name = "TestGenerator"
ierr = psspy.machine_data_4(ibus, gen_id, intgar_gen, realar_gen, gen_name)
if ierr == 0:
    print(f"Generator {gen_id} added successfully to bus {ibus}.")
else:
    print(f"Error adding generator {gen_id} to bus {ibus}. PSS®E error code: {ierr}")


 No power flow data changed for bus 25 [TESTBUS           ]
Bus 25 added successfully.

 No plant data changed at bus 25 [TESTBUS           ]
Plant data added successfully to bus 25.

 Messages for api MACHINE_DATA_4
 Messages for machine "1" at bus 25 [TESTBUS           ]:   (002601)
 Error: Voltage droop control "TESTGENERATOR" not found   (002622)

 Power flow data changed for machine "1" at bus 25 [TESTBUS           ]:
 X--ORIGINAL--X  X-NEW VALUE--X  DATA ITEM
          0               1      WMOD
          1               0      BASFLG
Error adding generator 1 to bus 25. PSS®E error code: -1


In [ ]:
realar = [0.0, 1.0, 0.0, ]